In [ ]:
!pip install mne
!pip install mambapy

In [23]:
import mne
import torch
import numpy as np
import torch.nn as nn
from mne import set_config
import torch.optim as optim
import torch.nn.functional as F
from mambapy.mamba import Mamba, MambaConfig
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

set_config('MNE_USE_CUDA', 'true')

In [24]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ieeg-data/sub-01_ses-iemu_task-film_acq-clinical_run-1_ieeg.vhdr
/kaggle/input/ieeg-data/sub-01_ses-iemu_task-film_run-1_events.tsv
/kaggle/input/ieeg-data/sub-01_ses-iemu_task-film_acq-clinical_run-1_ieeg.eeg
/kaggle/input/ieeg-data/sub-01_ses-iemu_task-film_acq-clinical_run-1_ieeg.vmrk
/kaggle/input/ieeg-data/sub-01_ses-iemu_acq-clinical_electrodes.tsv


# Pre Process

In [25]:
def prepare_ieeg_data(eeg_file, window_size=1000, overlap=500, apply_filter=False):

    raw = mne.io.read_raw_brainvision(eeg_file, preload=True)

    if apply_filter:
        raw.filter(l_freq=1, h_freq=200)  # Band-pass filter
        raw.notch_filter(freqs=[50, 60])   # Remove line noise
    
    data = raw.get_data()
    
    n_channels = data.shape[0]
    n_samples = data.shape[1]
    step = window_size - overlap
    
    n_windows = (n_samples - 2 * window_size) // step + 1
    
    X = np.zeros((n_windows, n_channels, window_size))
    y = np.zeros((n_windows, n_channels, window_size))
    
    for i in range(n_windows):
        start_x = i * step
        end_x = start_x + window_size
        
        start_y = end_x
        end_y = start_y + window_size
        
        X[i] = data[:, start_x:end_x]
        y[i] = data[:, start_y:end_y]
    
    return X, y, raw.ch_names

def extract_features(X):

    mean = np.mean(X, axis=2)
    std = np.std(X, axis=2)
    max_val = np.max(X, axis=2)
    min_val = np.min(X, axis=2)
    
    fft_vals = np.abs(np.fft.fft(X, axis=2))
    freq_mean = np.mean(fft_vals, axis=2)
    freq_std = np.std(fft_vals, axis=2)
    
    features = np.concatenate([
        mean, std, max_val, min_val,
        freq_mean, freq_std
    ], axis=1)
    
    return features

def prepare_for_modeling(eeg_file, test_size=0.2, random_state=42, feature_type='raw'):

    X, y, channels = prepare_ieeg_data(eeg_file)
    
    if feature_type == 'extracted':
        
        X = extract_features(X)
        y = extract_features(y)
        
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        
        X = scaler_X.fit_transform(X)
        y = scaler_y.fit_transform(y)
    else:  # 'raw'

        for i in range(X.shape[1]):
            scaler = StandardScaler()
            X[:, i, :] = scaler.fit_transform(X[:, i, :])
            y[:, i, :] = scaler.transform(y[:, i, :])

    train_idx = int(len(X) * (1 - test_size))
    
    X_train = X[:train_idx]
    X_test = X[train_idx:]
    y_train = y[:train_idx]
    y_test = y[train_idx:]
    
    return X_train, X_test, y_train, y_test, channels

def prepare_dataset (X_train, X_test):
    
    from torch.utils.data import TensorDataset, DataLoader
    
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    
    train_dataset = TensorDataset(X_train, X_train)
    test_dataset = TensorDataset(X_test, X_test)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    return train_loader, test_loader

# Pipeline

In [26]:
def train_model(
    model, train_loader, test_loader, criterion, optimizer, num_epochs, device
):
    model.to(device)
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            # Forward pass
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        
        # Evaluation phase
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                test_loss += loss.item()
        
        test_loss /= len(test_loader)
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")

    return model

# Models

In [27]:
config = MambaConfig(d_model=1000, n_layers=2)
model = Mamba(config)

# Main

In [28]:
X_train, X_test, y_train, y_test, channels = prepare_for_modeling(
    "/kaggle/input/ieeg-data/sub-01_ses-iemu_task-film_acq-clinical_run-1_ieeg.vhdr",
    feature_type='raw'  # or 'extracted' for feature-based prediction
  )

print()  
print(f"Training input shape: {X_train.shape}")
print(f"Training target shape: {y_train.shape}")
print(f"Test input shape: {X_test.shape}")
print(f"Test target shape: {y_test.shape}")
print(f"Number of channels: {len(channels)}")

Extracting parameters from /kaggle/input/ieeg-data/sub-01_ses-iemu_task-film_acq-clinical_run-1_ieeg.vhdr...
Setting channel info structure...
Reading 0 ... 860253  =      0.000 ...   420.045 secs...

Training input shape: (1373, 111, 1000)
Training target shape: (1373, 111, 1000)
Test input shape: (344, 111, 1000)
Test target shape: (344, 111, 1000)
Number of channels: 111


In [29]:
train_loader, test_loader = prepare_dataset(X_train, X_test)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [30]:
MambaModel = train_model(
    model, 
    train_loader, 
    test_loader, 
    criterion, 
    optimizer, 
    num_epochs=2, 
    device='cuda'
)

Epoch [1/2], Train Loss: 0.0004, Test Loss: 0.0000
Epoch [2/2], Train Loss: 0.0000, Test Loss: 0.0000


In [31]:
print(MambaModel)

Mamba(
  (layers): ModuleList(
    (0-1): 2 x ResidualBlock(
      (mixer): MambaBlock(
        (in_proj): Linear(in_features=1000, out_features=4000, bias=False)
        (conv1d): Conv1d(2000, 2000, kernel_size=(4,), stride=(1,), padding=(3,), groups=2000)
        (x_proj): Linear(in_features=2000, out_features=95, bias=False)
        (dt_proj): Linear(in_features=63, out_features=2000, bias=True)
        (out_proj): Linear(in_features=2000, out_features=1000, bias=False)
      )
      (norm): RMSNorm()
    )
  )
)
